In [968]:
%matplotlib inline
import io, os, sys, types

# this import imports the databaseloader script (which only does TGSS and VLSSR for now)
import Databaseloader as dl
import loadinvizerfit as lif

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import tkp.db
from astropy.coordinates import SkyCoord
from tkp.db.model import Varmetric
from tkp.db.model import Runningcatalog
from tkp.db.model import Newsource
from tkp.db.model import Extractedsource
from tkp.db.model import Image
import operator
from operator import itemgetter, attrgetter

import logging
import pandas as pd
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.orm import relationship
#import Tools
#import generic_tools
#import plotting_tools
from scipy.stats import norm
import pylab
pylab.rcParams['legend.loc'] = 'best'
from matplotlib.ticker import NullFormatter
from matplotlib.font_manager import FontProperties
#from astroML import density_estimation

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster
from scipy.spatial import distance
import scipy.stats as ss
import scipy.optimize as op
import scipy.integrate as si
import sys
!{sys.executable} -m pip install numpy
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import DatetimeTickFormatter
from bokeh.models import ColumnDataSource, Whisker
output_notebook()

logging.basicConfig(level=logging.INFO)

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Loading BokehJS ...

In [542]:
engine = 'postgresql'
host = 'vlo.science.uva.nl'
port = 5432
user = 'kmeulen'
password = 'kLu2oepRouv2UfoUPhoU'
database='KmeulenTrap4P23'
websiteURL = 'http://banana.transientskp.org/r4/vlo_'+database+'/runningcatalog/'
latexURL = '\url{http://banana.transientskp.org/r4/vlo_'+database+'/runningcatalog/'
latexHREF = '\href{http://banana.transientskp.org/r4/vlo_'+database+'/runningcatalog/'
query_loglevel = logging.WARNING  # Set to INFO to see queries, otherwise WARNING

In [543]:
# this is sqlalchemy script to login to the Banana database
logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)
db = tkp.db.Database(engine=engine, host=host, port=port,
                     user=user, password=password, database=database)
db.connect()
session = db.Session()

# Here i get the peak flux and the error on the peak flux from all the sources in the database
fpeak = session.query(Extractedsource.f_peak).all()
fpeake = session.query(Extractedsource.f_peak_err).all()
califreq = np.load("freqlist.npy")
califlux = np.load("fluxlist.npy")

INFO:tkp.db.database:connecting to database...
INFO:tkp.db.database:connected to: postgresql://kmeulen@vlo.science.uva.nl:5432/KmeulenTrap4P23


Query to find the variability parameters of the sources in a specific dataset 

In [544]:
# Here I specifiy which dataset of the database to use
dataset_id = 5
runcat_id = 16077

VarParams = session.query(Varmetric,Runningcatalog).select_from(join(Varmetric,Runningcatalog)).filter(Runningcatalog.dataset_id == dataset_id).all()

# VarFlux = session.query(Extractedsource).select_from(join(Extractedsource,Runningcatalog)).filter(Runningcatalog.dataset_id == dataset_id).all()



Make 2d array of all the sources except for the transients.

In [545]:

plotdata = [[VarParams[i].Runningcatalog.id, VarParams[i].Runningcatalog.wm_ra,\
             VarParams[i].Runningcatalog.wm_decl,VarParams[i].Runningcatalog.avg_ra_err,\
             VarParams[i].Runningcatalog.avg_decl_err, VarParams[i].Runningcatalog.datapoints,\
             session.query(Newsource).filter(Newsource.id==VarParams[i].Varmetric.newsource).one().newsource_type,\
             VarParams[i].Varmetric.newsource,VarParams[i].Varmetric.lightcurve_max,
             session.query(Newsource).filter(Newsource.id==VarParams[i].Varmetric.newsource).one().trigger_xtrsrc.id,VarParams[i].Varmetric.eta_int]\
            for i in range(len(VarParams)) if VarParams[i].Varmetric.newsource != None]


In [790]:
# The de Ruiter distance as shown in the Trap Paper section 4.4 https://arxiv.org/abs/1503.01526 

def CalcDeRuiter(transient,ra2,ra2e,dec2,dec2e):

    ra1 = transient['ra']
    dec1 = transient['dec']
    ra1e = transient['rae']
    dec1e = transient['dece']


    r=(np.sqrt((((-1*ra2 +ra1)**2)*(np.cos((dec1+dec2)/2))**2)/\
                       (ra1e**2+ra2e**2)+((-1*dec2+dec1)**2/(dec1e**2+dec2e**2))))
    return r 

In [547]:
# print len(VarParams)
for i in plotdata:
    if i[1] < 0:
        print i
print plotdata[0]

[16302, -175.406831965087, 46.9001121412271, 0.0203302739287593, 0.0138905737009884, 1, 0, 11999, 0.0428856457582911, 144492, 0.0]
[16306, -175.262038713443, 50.4435805115256, 0.0218095596273565, 0.0138890873903234, 1, 1, 11995, 1.1151067615734, 144478, 0.0]
[12138, -178.68710322285, 48.2800139594889, 0.0208757548373037, 0.0138913353101891, 1, 0, 8283, 0.230704566461283, 92142, 3.75259449568856]
[6953, -170.19853453585, 48.5828584030669, 0.0210081678123388, 0.0138907226744383, 1, 0, 2493, 0.175779895345452, 19167, 2.44340007325863]
[16415, -176.663620460539, 45.2724804260299, 0.0197396124745371, 0.0138908505411807, 1, 0, 12030, 0.138457014076291, 144604, 0.0]
[15385, -171.378627993494, 45.8812995456131, 0.0199543221373866, 0.013890558666435, 1, 0, 10922, 0.115545190781324, 128822, 1.67228951308787]
[12009, -175.652381489827, 45.3114571587702, 0.0197525331032056, 0.0138902232550875, 1, 0, 7666, 0.0713849106737846, 87820, 2.80962627063147]
[16446, -170.786254214696, 47.0065399649175, 0.0

In [789]:
# Comparing the fluxes heavily inspired on the de Ruiter distance as used by Bart's thesis: https://api-alumni.nl/media/uploads/theses/phd/lha-scheers-phd.pdf page 55

def CompareFluxes(transient,pflux2,pflux2e,scale=1):
    pflux1 = transient['pflux']*scale
    pflux1e = transient['pfluxe']*scale
    
    r = np.sqrt((((-1*pflux2 +pflux1)**2))/\
                       (pflux1e**2+pflux2e**2))
    return r

In [549]:
# calculates the frequency flux scale based on supplied califreq and califlux data

def CalcFreqScale(transient,datafreq,califreq,califlux):
    """Function for calculating the frequency scale"""
    transfreq = transient.freq
    for i in range(len(califreq)):
        if transfreq - 0.005 <= califreq[i] <= transfreq + 0.005:
            oldflux = califlux[i]
        if datafreq - 0.005 <= califreq[i] <= datafreq + 0.005:
            newflux = califlux[i]
    scale = newflux/oldflux
    return scale

def CalcRelFlux(transient,datafreq,dataflux):
    transfreq = transient.freq
    scale = np.power((transfreq/datafreq),-0.7)*dataflux 
    return scale

Function for pruning our candidate list

In [550]:
# def SelectorForCandidates(rcheck,ycheck,checklight,deruiter=True):
#     """Function for pruning the candidate list using both distance and lightcurve measurements"""
#     ctr = 0
#     print 'before:'
#     print len(Transients.instances)
#     oldlen = len(Transients.instances) + 1
    
# # #   Repeat until no improvement in the length of the candidate list
# #     while len(Transients.instances) - oldlen < 0:
#     ratlist = []
#     detclist = []

#     translist = []
# #         rlist = []
#     oldlen = len(Transients.instances)

# #         These lists are necessary to compare the candidate transients against each other
#     ralist = []
#     raelist = []
#     declist = []
#     decelist = []
#     pfluxlist = []
#     pfluxelist = []

# # """Check candidate transients against each other"""

#     print 'hoi'
#     for i in Transients.instances:
#         if not deruiter:
#             translist.append(i.radec)
#         else:
#             ralist.append(i.ra)
#             raelist.append(i.rae)
#             declist.append(i.dec)
#             decelist.append(i.dece)
#             pfluxlist.append(i.pflux)
#             pfluxelist.append(i.pfluxe)



# # #       if we are not using deruiter distance use euclidean (it is slightly outdated and not used alot
# # #       I would recommend just using the deruiterdistance method which also features the updated fluxcheckmethod)
# #     if not deruiter:
# #         index1list = []
# #         index2list = []
# #         translist = np.reshape(translist,(len(Transients.instances),2))
# #         Z = distance.cdist(translist,translist,'euclidean')

# #     # select on distance measure of 0.06 difference
# #         for g in range(len(Z)):
# #             for j in range(len(Z[g])):
# #                 if Z[g][j] <=0.06 and Z[g][j] !=0.0:
# #                     index1list.append(g)
# #                     index2list.append(j)
# #                 i = 0

# # # the deletion process for notderuiterdistance                    
# #         while i < len(index1list):

# #             if checklight:


# #                 try:
# #                     if  Transients.instances[index2list[i]].pflux*0.8 <= Transients.instances[index1list[i]].pflux <= Transients.instances[index2list[i]].pflux*1.2:
# #                         index1list.remove(index2list[i])
# #                         Transients.instances[index1list[i]] = []
# #                 except:
# #                     pass

# #             else:
# #                 index1list.remove(index2list[i])
# #                 Transients.instances[index1list[i]] = []


# #             i+=1

# #         i = 0
# #         while i < len(Transients.instances):
# #             if Transients.instances[i]:
# #                 pass
# #             else:
# #                 del Transients.instances[i]
# #                 i-=1     
# #             i+=1


# # #       "here we are using deRuiter to calculate the distances

# #     else:
# #         h = 0
# #         while h <(len(Transients.instances)):
# # #               calculate both the deruiterdistance's and compare the fluxes.
# #             r = CalcDeRuiter(vars(Transients.instances[h]),np.array(ralist),\
# #                              np.array(raelist),np.array(declist),np.array(decelist))
# # #           


# #             if checklight:
# #                 y = CompareFluxes(vars(Transients.instances[h]),np.array(pfluxlist),np.array(pfluxelist))
            
# # #             inbuild safe for if all sources are associated with each other
# #             jk = 0
# #             for j in range(len(r)):
# #                 if jk >= len(Transients.instances):
# #                     break
# #                 if checklight:
# #                     if r[j] <=rcheck and r[j]!= 0.0 and 0 < y[j] <=ycheck:
# #                         ratlist.append(Transients.instances[h].ra)
# #                         detclist.append(Transients.instances[h].dec)
# #                         del Transients.instances[h]
# #                         h-=1
# #                         jk=0
# #                         break
# #                 else:
                    

# #                     if r[j] <=rcheck:
# #                         ratlist.append(Transients.instances[h].ra)
# #                         detclist.append(Transients.instances[h].dec)
# # #                         print websiteURL + str(Transients.instances[h].id)
# #                         del Transients.instances[h]
# #                         h-=1
# #                         jk=0
# #                         break
# #                 jk+=1

# #             h+=1




#     # """Check Candidates against all sources"""
# #     a zero candidate that is being tracked if the flux is comparable to the transient 
# #     could be real things when it is consistently brighter or fainter than the candidate 1.
# #  Talk to Mark about pyc to get flux at specific coordinate ore look at the options. in the pyc-h 

#         masterindex = []
#         ilist = []
#         for i in Transients.instances:
#             indexlist = []
            
# #             again the outdated euclidean method
#             if not deruiter:
#                 Y = distance.cdist(i.radec,X,'euclidean')

#                 for g in Y:
#                     for j in range(len(g)):
#                         if checklight:
#                             if g[j] <=0.06 and 0.8*lightcata[j]<=i.pflux <= 1.2*lightcata[j]:
#                                 indexlist.append(j)
#                         else:
#                             if g[j] <=0.06:
#                                 indexlist.append(j)
                
# #             the de Ruiter method 
#             else:
#                 r = CalcDeRuiter(vars(i),np.array(racata),np.array(raecata),np.array(deccata),np.array(dececata))
#                 y = CompareFluxes(vars(i),np.array(lightcata),np.array(lightecata))
#                 for j in range(len(r)):
#                     if checklight:
#                         if r[j] <=rcheck and y[j]<=ycheck and r[j]!=0.0:
#                             indexlist.append(j)
                                
                            
                            
#                     else:
#                         if r[j] <=rcheck and r[j]!=0.0:
#                             indexlist.append(j)
#             masterindex.append(indexlist)
            
# # removing the fake candidates
#         i = 0
#         j= 0
#         while i < len(masterindex):
#             if masterindex[i]:
#                 del Transients.instances[j]
#                 j-=1
#             i+=1
#             j+=1

#     print 'after:'    
#     print len(Transients.instances)

#     zxcv = np.dstack((np.array(ratlist),np.array(detclist)))
#     print zxcv

Function for comparing transient candidate against external databases

In [733]:

def CheckOutsideDatabase(data,sources,rcheck,ycheck,checklight,gamma,Cali,deruiter = True,euccutoff = 0.06):
    """ Function for checking against outside database the data entry is for the database rcheck is condition \
where we check deruiter against, ycheck is where we check de flux against and checklight,gamma,deruiter\
are booleans which switch on checking the flux(checklight), checking using error on the flux(gamma) and \
using deruiter distance

"""
    if Cali:
        scale =  CalcFreqScale(sources.instances[0],data.freq,califreq,califlux)
    else:
        scale = CalcRelFlux(sources.instances[0],data.freq,data.pflux)
    masterindex = []
    savedilist = []
    print 'before:'
    

    print len(sources.instances)

    for i in sources.instances:
        indexlist = []

        if not deruiter:
            Y = distance.cdist(i.radec,data.radec,'euclidean')
            if gamma:
                y = CompareFluxes(vars(i),np.array(data.pflux),np.array(data.pfluxe),scale)
                
                for g in Y:

                    for j in range(len(g)):

                        if g[j] <=euccutoff and y[j]>= ycheck:
                            if i.id not in savedilist:
                                FluxVar(vars(i),j,data,scale)
                                savedilist.append(i.id)
                            indexlist.append(j)

                        elif g[j] <=euccutoff:
                            indexlist.append(j)
                    
            else:
                for g in Y:

                    for j in range(len(g)):
                        if checklight:

                            if g[j] <=euccutoff and (i.pflux*scale <= 0.9*data.pflux[j] or i.pflux*scale>=1.1*data.pflux[j]):
                                if i.id not in savedilist:
                                    FluxVar(vars(i),j,data,scale)
                                    savedilist.append(i.id)
                                indexlist.append(j)

                            elif g[j] <=euccutoff:
                                indexlist.append(j)
                        else:
                            if g[j] <=euccutoff:
                                indexlist.append(j)

        else:

            r = CalcDeRuiter(vars(i),np.array(data.ra),np.array(data.rae),np.array(data.dec),np.array(data.dece))
            if gamma:
                y = CompareFluxes(vars(i),np.array(data.pflux),np.array(data.pfluxe),scale)

                for j in range(len(r)):

                    if r[j] <=rcheck and y[j]>= ycheck:
                        indexlist.append(j)
#                         if i.id not in savedilist:
                        FluxVar(vars(i),j,data,scale)
                        print j
                        print i.pflux
                        print i.pflux *scale
                        print data.pflux[j]
                        print data.ra[j]
                        print data.dec[j]
                        savedilist.append(i.id)

                    elif r[j] <=rcheck:
                        indexlist.append(j)

            else:
                for j in range(len(r)):
                    if checklight:
                        if r[j] <=rcheck and (i.pflux*scale <= 0.9*data.pflux[j] or i.pflux*scale>=1.1*data.pflux[j]):
                            indexlist.append(j)
                            if i.id not in savedilist:
                                FluxVar(vars(i),j,data,scale)
                                savedilist.append(i.id)
                        elif r[j] <=rcheck:
                            indexlist.append(j)





                    else:
                        if r[j] <=rcheck:
                            indexlist.append(j)

        masterindex.append(indexlist)
    
        
    i = 0
    while i < len(masterindex):
        if not masterindex[i]:
            PosVar(vars(sources.instances[i]),data)
        i+=1

    print 'after:'  
    print str(len(PosVar.instances)) +" Interesting candidates"
    print str(len(FluxVar.instances)) + " Flux Varying candidates"


In [949]:

def CheckSimOutsideDatabase(data,sources,rcheck,ycheck,checklight,gamma,Cali,deruiter = True,euccutoff = 0.06):
    """ Function for checking against outside database the data entry is for the database rcheck is condition \
where we check deruiter against, ycheck is where we check de flux against and checklight,gamma,deruiter\
are booleans which switch on checking the flux(checklight), checking using error on the flux(gamma) and \
using deruiter distance

"""
    masterindex = []
    savedilist = []
    print 'before:'
    if Cali:
        scale =  CalcFreqScale(sources[0],data.freq,califreq,califlux)
    else:
        scale = CalcRelFlux(sources[0],data.freq,data.pflux)

    print len(sources)

    for ctr,i in enumerate(sources):
        indexlist = []

        if not deruiter:
            Y = distance.cdist(i.radec,data.radec,'euclidean')
            if gamma:
                y = CompareFluxes(i,np.array(data.pflux),np.array(data.pfluxe),scale)
                
                for g in Y:

                    for j in range(len(g)):

                        if g[j] <=euccutoff and y[j]>= ycheck:
                            if i.id not in savedilist:
                                FluxVar(vars(i),j,data,scale)
                                savedilist.append(i.id)
                            indexlist.append(j)

                        elif g[j] <=euccutoff:
                            indexlist.append(j)
                    
            else:
                for g in Y:

                    for j in range(len(g)):
                        if checklight:

                            if g[j] <=euccutoff and (i.pflux*scale <= 0.9*data.pflux[j] or i.pflux*scale>=1.1*data.pflux[j]):
                                if i.id not in savedilist:
                                    FluxVar(vars(i),j,data,scale)
                                    savedilist.append(i.id)
                                indexlist.append(j)

                            elif g[j] <=euccutoff:
                                indexlist.append(j)
                        else:
                            if g[j] <=euccutoff:
                                indexlist.append(j)

        else:

            r = CalcDeRuiter(vars(i),np.array(data.ra),np.array(data.rae),np.array(data.dec),np.array(data.dece))
            if gamma:
                y = CompareFluxes(vars(i),np.array(data.pflux),np.array(data.pfluxe),scale)

                for j in range(len(r)):

                    if r[j] <=rcheck:
                        indexlist.append(j)
               
                if checklight:
                    pfluxlist = []
                    pfluxelist = []
                    for z in indexlist:
                        pfluxlist.append(data.pflux[z])
                        pfluxelist.append(data.pfluxe[z])

                    y = CompareFluxes(vars(i),np.array(pfluxlist),np.array(pfluxelist))
                    
                    for o in range(len(y)):
                        if y[o]>=ycheck:
                            FluxVar(vars(i),indexlist[o],data,scale)
                            break

            else:
                for j in range(len(r)):
                    if checklight:
                        if r[j] <=rcheck and (i.pflux*scale <= 0.9*data.pflux[j] or i.pflux*scale>=1.1*data.pflux[j]):
                            indexlist.append(j)
                            if i.id not in savedilist:
                                FluxVar(vars(i),j,data,scale)
                                savedilist.append(i.id)
                        elif r[j] <=rcheck:
                            indexlist.append(j)





                    else:
                        if r[j] <=rcheck:
                            indexlist.append(j)

        masterindex.append(indexlist)
    
#     check if thereis a zero listentry in masterindex.. if so append it to the varying pos sources class.
    i = 0
    print masterindex
    while i < len(masterindex):
        if not masterindex[i]:
            PosVar(vars(sources[i]),data)
        i+=1

    print 'after:'  
    print str(len(PosVar.instances)) +" Interesting candidates"
    print str(len(FluxVar.instances)) + " Flux Varying candidates"


Class object for saving our candidate transients, class object for storing objects that have varying flux compared to external database and
Initialize our transients class and other banana sources lists.

(This piece of code has to be rerun everytime you adjust something)

In [983]:

class Transients(object):
    instances = []
    def __init__(self,ra,rae,dec,dece,ids,pflux,pfluxe):
        self.id = ids
        y = np.stack((ra,dec),axis = -1)
        self.ra = ra
        self.rae = rae
        self.dec = dec
        self.dece = dece
        self.pflux = pflux
        self.pfluxe = pfluxe
        self.freq = 144
        self.keys = ['yoloswag']
        self.radec = np.reshape(y,(1,2))
        c = SkyCoord(self.ra, self.dec, frame='fk5', unit='deg')
        self.fk5 = c.to_string('hmsdms',sep=' ',precision= 0)
        Transients.instances.append(self)

    def DelFalse(i):
        del Transients.instances[i]

class PosVar(object):
    """class object for storing objects that have a varying position compared to the database we are looking in"""
    instances = []
    def __init__(self,params,database):
        self.id = params['id']            
        self.ra = params['ra']
        self.database = database
        self.dataname = database.name
        self.dec = params['dec']
        self.radec = params['radec']
        self.pflux = params['pflux']
        c = SkyCoord(params['ra'], params['dec'], frame='fk5', unit='deg')
        self.fk5 = c.to_string('hmsdms',sep=' ',precision= 0)
        try:
            self.iflux = params['iflux']
        except:
            pass
        PosVar.instances.append(self)
        

class TestInstance(object):
    """class object for storing objects that have a varying position compared to the database we are looking in"""
    instances = []
    def __init__(self,params):
        self.id = params['id']            
        self.ra = params['ra']
        self.dec = params['dec']
        self.rae = params['rae']
        self.dece = params['dece']
        self.radec = params['radec']
        self.pflux = params['pflux']
        self.freq = params['freq']
        self.pfluxe = params['pfluxe']
        c = SkyCoord(params['ra'], params['dec'], frame='fk5', unit='deg')
        self.fk5 = c.to_string('hmsdms',sep=' ',precision= 0)
        try:
            self.iflux = params['iflux']
        except:
            pass
        TestInstance.instances.append(self)
        
class SavedFluxSources(object):
    """class object for storing objects that have a varying position compared to the database we are looking in"""
    instances = []
    def __init__(self,initialid,initialpflux,initialpfluxerr,params):
        self.id = params['id']            
        self.ra = params['ra']
        self.dec = params['dec']
        self.rae = params['rae']
        self.dece = params['dece']
        self.radec = params['radec']
        self.pflux = params['pflux']
        self.freq = params['freq']
        self.pfluxe = params['pfluxe']
        c = SkyCoord(params['ra'], params['dec'], frame='fk5', unit='deg')
        self.fk5 = c.to_string('hmsdms',sep=' ',precision= 0)
        self.cmpid = initialid
        self.cmppflux = initialpflux
        self.cmppfluxerr = initialpfluxerr
        try:
            self.iflux = params['iflux']
        except:
            pass
        SavedFluxSources.instances.append(self)

class SavedPosSources(object):
    """class object for storing objects that have a varying position compared to the database we are looking in"""
    instances = []
    def __init__(self,params):
        self.id = params['id']            
        self.ra = params['ra']
        self.dec = params['dec']
        self.rae = params['rae']
        self.dece = params['dece']
        self.radec = params['radec']
        self.pflux = params['pflux']
        self.freq = params['freq']
        self.pfluxe = params['pfluxe']
        c = SkyCoord(params['ra'], params['dec'], frame='fk5', unit='deg')
        self.fk5 = c.to_string('hmsdms',sep=' ',precision= 0)
        try:
            self.iflux = params['iflux']
        except:
            pass
        SavedPosSources.instances.append(self)  
    
class FluxVar(object):
    """class object for storing objects that have a varying flux compared to the database we are looking in"""
    instances = []
    def __init__(self,params,databaseentry,database,scale):
        self.id = params['id']
        self.ra = params['ra']
        self.database = database
        self.dataname = database.name
        self.scale = scale
        self.dec = params['dec']
        self.radec = params['radec']
        self.pflux = params['pflux']
        self.entry = databaseentry
        c = SkyCoord(params['ra'], params['dec'], frame='fk5', unit='deg')
        c = c.icrs
        self.fk5 = c.to_string('hmsdms',sep=' ',precision= 0)
        try:
            self.iflux = params['iflux']
        except:
            pass
        FluxVar.instances.append(self)
    

        
racata = []
raecata = []
deccata = []
dececata = []
idcata = []
lightcata = []
lightecata = []
for i in range(len(plotdata)):
    if plotdata[i][6] == 1:
        Transients(plotdata[i][1],plotdata[i][3],plotdata[i][2],plotdata[i][4],plotdata[i][0],fpeak[plotdata[i][9]-1][0],fpeake[plotdata[i][9]-1][0])
    else:
        racata.append(plotdata[i][1])
        raecata.append(plotdata[i][3])
        deccata.append(plotdata[i][2])
        dececata.append(plotdata[i][4])
        idcata.append(plotdata[i][0])
        lightcata.append(fpeak[plotdata[i][9]-1][0])
        lightecata.append(fpeake[plotdata[i][9]-1][0])
X = np.stack((racata,deccata),axis = -1)

racata = np.array(racata)
raecata = np.array(raecata)
dececata = np.array(dececata)
deccata = np.array(deccata)
idcata = np.array(idcata)
lightcata = np.array(lightcata)
lightecata = np.array(lightecata)


# Transients(183,0.01,45,0.01,99999999,1.32,0.001)

In [984]:
rasl = []
raesl = []
decsl = []
decesl = []
idsl = []
pfluxsl = []
pfluxesl = []
for i in range(len(plotdata)):
    rasl.append(plotdata[i][1])
    raesl.append(plotdata[i][3])
    decsl.append(plotdata[i][2])
    decesl.append(plotdata[i][4])
    idsl.append(plotdata[i][0])
    pfluxsl.append(fpeak[plotdata[i][9]-1][0])
    pfluxesl.append(fpeake[plotdata[i][9]-1][0])



raseries = pd.Series(racata)
raseries
ras = pd.Series(rasl, name='ra')
raes = pd.Series(raesl, name='raerr')
decs = pd.Series(decsl, name='dec')
deces = pd.Series(decesl, name='decerr')
ids = pd.Series(idsl, name='id')
pfluxs = pd.Series(pfluxsl, name='pflux')
pfluxes = pd.Series(pfluxesl, name='pfluxerr')

pandacata = pd.concat([ras, raes,decs,deces,ids,pfluxs,pfluxes], axis=1)
# for i in Transients.instances:
#     if 189.8 < i.ra < 190:
#         print i.ra

here we call the function for calculating the de ruiter distance between candidates and our found sources and if True is given we also check if the max of the lightcurve of the sources compared fall withing 5 sigma flux of each other (thus making it increasingly likely for   the source to be a sidelobe)

This extra flux comparison is used in order to not potentially write of a faint or bright transient close to a known source.

In [985]:
# SelectorForCandidates(rcheck = 3, ycheck = 5.68,checklight = False, deruiter = True)

In [986]:
def SelectorForCandidateslist(rcheck,ycheck,checklight,deruiter=True):
    """Function for pruning the candidate list using both distance and lightcurve measurements"""
    ctr = 0
    print 'before:'
    print len(Transients.instances)
    oldlen = len(Transients.instances) + 1
    
# #   Repeat until no improvement in the length of the candidate list
#     while len(Transients.instances) - oldlen < 0:
    ratlist = []
    detclist = []

    translist = []
#         rlist = []
    oldlen = len(Transients.instances)

#         These lists are necessary to compare the candidate Transientss against each other
    ralist = []
    raelist = []
    declist = []
    decelist = []

# """Check candidate Transientss against each other"""

    for h in Transients.instances:
        if not deruiter:
            translist.append(h.radec)
        else:
            ralist.append(h.ra)
            raelist.append(h.rae)
            declist.append(h.dec)
            decelist.append(h.dece)
            

    masterindex = []
    savedlist = []
    first = True
    for i in Transients.instances:
        nocount = False
        indexlist = []


# #             again the outdated euclidean method
#         if not deruiter:
#             Y = distance.cdist(i.radec,X,'euclidean')

#             for g in Y:
#                 for j in range(len(g)):
#                     if checklight:
#                         if g[j] <=0.06 and 0.8*lightcata[j]<=i.pflux <= 1.2*lightcata[j]:
#                             indexlist.append(j)
#                     else:
#                         if g[j] <=0.06:
#                             indexlist.append(j)

#             the de Ruiter method 
#         else:
        r = CalcDeRuiter(vars(i),np.array(ralist),\
        np.array(raelist),np.array(declist),np.array(decelist))
        y = CompareFluxes(vars(i),np.array(lightcata),np.array(lightecata))
        for j in range(len(r)):

            if r[j] <=rcheck and r[j]!=0.0:
                indexlist.append(j)
                nocount = True
                        
#         SAVE ENTRIES THAT HAVE A BIG DIFFERENCE ON FLUX
        if checklight:
            pfluxlist = []
            pfluxelist = []
            for z in indexlist:            
                pfluxlist.append(Transients.instances[z].pflux)
                pfluxelist.append(Transients.instances[z].pfluxe)

            y = CompareFluxes(vars(i),np.array(pfluxlist),np.array(pfluxelist))
            
            if first:
                print y
                first = False
                try:
                    print Transients.instances[indexlist[0]].id
                except:
                    pass
            for o in range(len(y)):
                if y[o]>=ycheck:
                    SavedFluxSources(i.id,i.pflux,i.pfluxe,vars(Transients.instances[indexlist[o]]))
                    break
                    
                 
#         DELETE DOUBLE ENTRIES reform lists
        if nocount:
            k = 0
            g = 0
            while k < len(indexlist):
                del Transients.instances[indexlist[k]-g]

                k+=1
                g+=1

            ralist = []
            raelist = []
            declist = []
            decelist = []

            for h in Transients.instances:
                ralist.append(h.ra)
                raelist.append(h.rae)
                declist.append(h.dec)
                decelist.append(h.dece)


    print 'after:'
#     add back the no duplicate detections
#     for i in SavedPosSources.instances:
#         TestInstance(vars(i))
    print len(Transients.instances)
    print len(SavedFluxSources.instances)



In [987]:
for i in Transients.instances:
    TestInstance(vars(i))

In [988]:
print len(TestInstance.instances)
TestInstance.instances =[]
SavedFluxSources.instances=[]
SavedPosSources.instances=[]
print len(TestInstance.instances)

602
0


In [989]:
SelectorForCandidateslist(rcheck = 3, ycheck = 3,checklight = True, deruiter = True)


before:
602
[72.40047787 66.67361514 73.49158554  8.98839618  7.60474333 10.56913633
 74.32682577 62.16881908  6.09566687 10.12399222  2.30997548 10.40042313
 13.51870588 73.09609369  7.34464434 10.59262606 76.8032959   6.27464459
 71.31617417  1.64595361 72.85310593 78.80836623  9.4549157  66.28679245
 79.32491235 78.35575131 77.22965829 11.28355494 69.44672618  3.39277634
 70.23387871]
15380
after:
50
37


In [990]:
# 189.857276545
# print "Candidate links"
# for i in TestInstance.instances:
#     print websiteURL+str(i.id)

In [991]:
# 189.857276545
# print "Candidate links"
# for i in SavedFluxSources.instances:
#     print websiteURL+str(i.id)

In [992]:
# 189.857276545
# print "Candidate links"
# for i in SavedPosSources.instances:
#     print websiteURL+str(i.id)

In [993]:
# 189.857276545
# print "Candidate links"
# for i in Transients.instances:
#     print websiteURL+str(i.id)

In [994]:
# for i in Transients.instances:
#     if 189.8 < i.ra < 190:
#         print i.ra


print out in latex format:

In [995]:
# for i in Transients.instances:
#     print str(i.id) + " " + "&"+ " " + '%0.3f' %i.ra + " " + "&"+ " "+ "%0.3f" %(i.dec) +\
#     " " + "&" + " " +latexURL +str(i.id)+"}" +" " + '&' + "  "+"\\" + "\\"

In [996]:
try:
    print SavedFluxSources.instances[0].pflux
    print Transients.instances[0].pflux
except:
    pass
FluxVar.instances = []
PosVar.instances = []

0.717379657755
7.18846941692


# tgss 
is the object which has tggs survey data for the field of P23 loaded into it.
could probably just automate it here but lets leave it for now

In [997]:
tgsslist = dl.ReadData("tgss")
tgss = tgsslist[1]
print tgss.keys
print tgss.freq
print tgss.name

['name', 'radec', 'ra', 'rae', 'dec', 'dece', 'pflux', 'pfluxe', 'freq']
147.5
tgss


In [998]:
CheckSimOutsideDatabase(tgss,Transients.instances,rcheck = 3,ycheck = 3,\
                     checklight = True, gamma = True,Cali = True,deruiter = True,euccutoff = .06)

before:
50
[[249], [11], [885, 950, 982], [735], [], [0, 1, 2, 93, 106, 281, 297, 389, 732, 985, 1315, 1341, 2195, 5000, 5018, 6227, 10891, 16501, 24935, 46974, 63364, 65144, 77734], [234], [274], [251], [125], [393], [665], [802], [1062], [95, 103, 137, 174], [346], [417], [263, 271, 280], [114, 117, 120, 123], [392, 411], [183], [517], [388, 398], [976], [287], [], [451], [958], [257], [127], [678], [405], [363], [486], [809], [], [621, 647], [615], [220, 265, 266], [228, 229], [753], [341], [246], [569], [1072], [368], [], [1085], [175], []]
after:
5 Interesting candidates
22 Flux Varying candidates


In [943]:
print Transients.instances[0].fk5


12 22 52 +50 26 56


## VLSSr

In [952]:
# Probably need to pip install xlrd for excell support in pandas
vlssr = dl.ReadData('vlssr')[0]
print vlssr.keys
print vlssr.freq

vlssrdataset.xls
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
['name', 'radec', 'ra', 'rae', 'dec', 'dece', 'pflux', 'pfluxe', 'freq']
74


In [953]:
CheckOutsideDatabase(vlssr,Transients,rcheck = 3 ,ycheck = 3,checklight = True,\
                     gamma = True,Cali=True,deruiter = False,euccutoff = 0.06)

before:
50
after:
10 Interesting candidates
51 Flux Varying candidates


In [955]:

savedpos = []
savedflux = []
print len(PosVar.instances)
print len(FluxVar.instances)
for i in PosVar.instances:
    if i.id not in savedpos:
        savedpos.append(i.id)
for i in FluxVar.instances:
    if i.id not in savedflux:
        savedflux.append(i.id)
        
print len(savedpos)
print len(savedflux)

10
51
5
37


## Print saved flux sources LATEX

In [573]:
print "\\begin{longtable}{c|c|c|c|c|c|c|c|c|c} \n\
\\toprule \n Id & Ra (deg) & Dec (deg) & Fk5 & Flux & Fluxerr & cmpId & cmpFlux & cmpFluxerr & Candidate? (y/n)  \\\\\\midrule \n\\endfirsthead \n\\toprule \
\n Id & Ra (deg) & Dec (deg) & Fk5 & Flux & Fluxerr & cmpId & cmpFlux & cmpFluxerr & Candidate? (y/n)  \\\\\\midrule \n\\endhead \n\\bottomrule \n\\endfoot \n\\bottomrule"

# define caption for table
print "\\caption{ SAVED FLUX SOURCES} \n\\endlastfoot"
for i in SavedFluxSources.instances:
    print latexHREF + str(i.id) +"}{%i}" %(i.id)+ " " + "&"+ " " + '%0.3f' %i.ra + " " + "&"+ " "+ "%0.3f" %(i.dec) +\
    " " + "&" + i.fk5+"&" + " " + '%0.3f' %(i.pflux) +"&" + " " + '%0.3f' %(i.pfluxe) +"&" + " " +\
    latexHREF + str(i.cmpid) +"}{%i}" %(i.cmpid)\
    +"&" + " " +'%0.3f' %(i.cmppflux) +"&" + " "\
    + '%0.3f' %(i.cmppfluxerr) +"&" + " " +"\\" + "\\"
print "\\end{longtable}"

\begin{longtable}{c|c|c|c|c|c|c|c|c|c} 
\toprule 
 Id & Ra (deg) & Dec (deg) & Fk5 & Flux & Fluxerr & cmpId & cmpFlux & cmpFluxerr & Candidate? (y/n)  \\\midrule 
\endfirsthead 
\toprule 
 Id & Ra (deg) & Dec (deg) & Fk5 & Flux & Fluxerr & cmpId & cmpFlux & cmpFluxerr & Candidate? (y/n)  \\\midrule 
\endhead 
\bottomrule 
\endfoot 
\bottomrule
\caption{ SAVED FLUX SOURCES} 
\endlastfoot
\href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/15380}{15380} & 185.695 & 50.465 &12 22 47 +50 27 53& 0.717& 0.066& \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16077}{16077}& 7.188& 0.061& \\
\href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/13300}{13300} & 184.777 & 48.499 &12 19 06 +48 29 56& 0.770& 0.004& \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/15556}{15556}& 0.819& 0.005& \\
\href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/12626}{12626} & 188.278 & 45.483 &1

# Print the same position Fluxvariating sources

In [944]:
for i in FluxVar.instances:
    
    print "Data compared to " + i.database.name
    print i.id
    print websiteURL+str(i.id)
    print 'bananaflux: ' + str(i.pflux*i.scale)
    print i.database.name+'flux: ' + str(i.database.pflux[i.entry])
    print i.database.ra[i.entry]
    print i.database.dec[i.entry]
    print "------------------------"
        
#     if i.database == 'vssr':
        
#         print 'bananaflux: ' + str(i.pflux * vlssrscale)
#         print 'vlssrflux: ' + str(vssr.pflux[i.entry])
#         print "########################"
#     print vssr.ra[i.entry]
#     print vssr.dec[i.entry]
#     print i.ra
#     print i.dec

Data compared to tgss
10999
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10999
bananaflux: 2.033016413901846
tgssflux: 0.0429
185.23462
48.70758
------------------------
Data compared to tgss
10309
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10309
bananaflux: 0.9004814823173248
tgssflux: 0.5919
185.36694
50.8145
------------------------
Data compared to tgss
10500
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10500
bananaflux: 0.8996061068708251
tgssflux: 0.0207
188.10638
48.36139
------------------------
Data compared to tgss
12944
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/12944
bananaflux: 1.5521627763364672
tgssflux: 0.06409999999999999
183.42596
48.391420000000004
------------------------
Data compared to tgss
6567
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/6567
bananaflux: 0.7236795389339572
tgssflux: 0.4786
186.30178999999998
45.391329999999996
------------

# Print the same position Fluxvariating sources in Latex ready format

In [999]:

# SEPERATE DATABASES
databaselist = ['tgss','vlssr']

samesieslist = []
for i in range(len(FluxVar.instances)):
    samesie = False
    tempdict = {'tgss':None,'vlssr':None,'id':None,'ra':None,'dec':None,'fk5':None}
    
#   create combined name if entry both in vlssr and tgss
    for j in range(len(samesieslist)):
        if FluxVar.instances[i].id == samesieslist[j]['id']:
#             samesieslist[j].append(FluxVar.instances[i])
            samesieslist[j][FluxVar.instances[i].database.name] = FluxVar.instances[i]
          
            samesie = True
    if not samesie:
        tempdict[FluxVar.instances[i].database.name] = FluxVar.instances[i]
        tempdict['id'] = FluxVar.instances[i].id
        tempdict['ra'] = FluxVar.instances[i].ra
        tempdict['dec'] = FluxVar.instances[i].dec
        tempdict['fk5'] = FluxVar.instances[i].fk5
        samesieslist.append(tempdict)


print "\\begin{longtable}{c|c|c|c|c|c|c|c|c|c|c|c|c} \n\
\\toprule \nFlux147.5MHz (Jy) & Tgss (Jy) & Flux74MHz (Jy)& Vlssr (Jy) & fk5 & Ra (deg) & Dec (deg) &tggsRa&tgssDec&\
vlssrRa&vlssrDec& Id/Link & (y/n)  \\\\\\midrule \n\\endfirsthead \n\\toprule \
\nFlux147.5MHz (Jy) & Tgss (Jy) & Flux74MHz (Jy)& Vlssr (Jy)& fk5 & Ra (deg) & Dec (deg)  &tggsRa&tgssDec&\
vlssrRa&vlssrDec& Id/Link & (y/n) \\\\\\midrule \n\\endhead \n\\bottomrule \n\\endfoot \n\\bottomrule"

# define caption for table
print "\\caption{ Comparison of variable fluxes with the same position for different databases 1hr} \n\\endlastfoot"
lastname = None
# samesieslist = sorted(samesieslist, key=lambda e: (e['ra'], e['dec']))
for i in samesieslist:
    a= ''
    b=''
    for j in(databaselist):
        try: 
            a+= str(np.around(i[j].pflux*i[j].scale,3)) + " " + "&"+ " "
        except:
            a += "-" +" "+ "&"+ " "
        try:
            a+= str(np.around(i[j].database.pflux[i[j].entry],3)) + " " + "&"+ " "   
        except:
            a += "-" +" "+ "&"+ " "
            
        try: 
            b+= '%0.3f' %(i[j].database.ra[i[j].entry])+" "+"&"+" "
            b+= '%0.3f' %(i[j].database.dec[i[j].entry])+" "+"&"+" "
        except:
            b+= "-" + " " + "&"+" " "-" + " " + "&"+" "
        
    a +=i['fk5'] + " " + '&' + "  "+ '%0.3f' %i['ra'] + " " + "&"+ " "+ "%0.3f" %i['dec'] \
    + " " + '&' + "  "+ b+ latexHREF +str(i["id"])+"}" + "{%i}" %i['id'] +" "+"&"+" "+  "\\" + "\\"
    print a

print "\\end{longtable}"

\begin{longtable}{c|c|c|c|c|c|c|c|c|c|c|c|c} 
\toprule 
Flux147.5MHz (Jy) & Tgss (Jy) & Flux74MHz (Jy)& Vlssr (Jy) & fk5 & Ra (deg) & Dec (deg) &tggsRa&tgssDec&vlssrRa&vlssrDec& Id/Link & (y/n)  \\\midrule 
\endfirsthead 
\toprule 
Flux147.5MHz (Jy) & Tgss (Jy) & Flux74MHz (Jy)& Vlssr (Jy)& fk5 & Ra (deg) & Dec (deg)  &tggsRa&tgssDec&vlssrRa&vlssrDec& Id/Link & (y/n) \\\midrule 
\endhead 
\bottomrule 
\endfoot 
\bottomrule
\caption{ Comparison of variable fluxes with the same position for different databases 1hr} 
\endlastfoot
7.069 & 4.935 & - & - & 12 22 52 +50 26 56 &  185.716 & 50.449 &  185.716 & 50.448 & - & - & \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16077}{16077} & \\
0.91 & 0.151 & - & - & 12 33 07 +45 29 02 &  188.278 & 45.484 &  187.839 & 45.473 & - & - & \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/8007}{8007} & \\
2.033 & 0.043 & - & - & 12 21 11 +48 42 40 &  185.295 & 48.711 &  185.235 & 48.708 & - & - & \hr

# Print the Position variating sources in Latex ready format

In [576]:
# SEPERATE DATABASES

print "\\begin{longtable}{c|c|c|c|c|c} \n\
\\toprule \nDatabase & Ra (deg) & Dec (deg) & Fk5 & Id/Link & Candidate? (y/n)  \\\\\\midrule \n\\endfirsthead \n\\toprule \
\nDatabase & Ra (deg) & Dec (deg) & Fk5 & Id/Link & Candidate? (y/n)  \\\\\\midrule \n\\endhead \n\\bottomrule \n\\endfoot \n\\bottomrule"

# define caption for table
print "\\caption{ Comparison position of candidates with different databases for the 10 min interval data} \n\\endlastfoot"
lastname = None
for i in PosVar.instances:
    if i.database.name != lastname and lastname:
        print "\\hline"
    print i.database.name +" "+ "&"+ " " + '%0.3f' %i.ra + " " + "&"+ " "+ "%0.3f" %i.dec +\
    " " + "&" + " " + i.fk5 +" " + '&' + "  "+ latexHREF +str(i.id)+"}" + "{%i}" %i.id +" " + '&' + "  "+"\\" + "\\"
    
    lastname = i.database.name

print "\\end{longtable}"

\begin{longtable}{c|c|c|c|c|c} 
\toprule 
Database & Ra (deg) & Dec (deg) & Fk5 & Id/Link & Candidate? (y/n)  \\\midrule 
\endfirsthead 
\toprule 
Database & Ra (deg) & Dec (deg) & Fk5 & Id/Link & Candidate? (y/n)  \\\midrule 
\endhead 
\bottomrule 
\endfoot 
\bottomrule
\caption{ Comparison position of candidates with different databases for the 10 min interval data} 
\endlastfoot
tgss & -175.262 & 50.444 & 12 18 57 +50 26 37 &  \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16306}{16306} &  \\
tgss & -174.211 & 44.590 & 12 23 09 +44 35 24 &  \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16326}{16326} &  \\
tgss & -171.721 & 45.481 & 12 33 07 +45 28 52 &  \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16375}{16375} &  \\
tgss & -173.524 & 44.939 & 12 25 54 +44 56 22 &  \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16130}{16130} &  \\
tgss & -176.475 & 50.479 & 12 14 06 +50

In [957]:
# COMBINED DATABASES (SLOWER)

samesieslist = []
for i in range(len(PosVar.instances)):
    samesie = False
    
#   create combined name if entry both in vlssr and tgss
    for j in range(len(samesieslist)):
        if PosVar.instances[i].id == samesieslist[j][0]:
            samesieslist[j][1] = samesieslist[j][1] + " \& " + PosVar.instances[i].database.name
            samesie = True
    if not samesie:
        samesieslist.append([PosVar.instances[i].id,PosVar.instances[i].database.name,PosVar.instances[i]])
    
samesieslist = sorted(samesieslist, key=itemgetter(1))

print "\\begin{longtable}{c|c|c|c|c|c} \n\
\\toprule \nDatabase & Ra (deg) & Dec (deg)& Fk5 & Id/Link & Candidate? (y/n)  \\\\\\midrule \n\\endfirsthead \n\\toprule \
\nDatabase & Ra (deg) & Dec (deg) & Fk5 &Id/Link & Candidate? (y/n)  \\\\\\midrule \n\\endhead \n\\bottomrule \n\\endfoot \n\\bottomrule"

# define caption for table
print "\\caption{ Comparison position of candidates with different databases for the 1hr interval data} \n\\endlastfoot"
lastname = None
for i in samesieslist:
    if i[1] != lastname and lastname:
        print "\\hline"
    print i[1] +" "+ "&"+ " "+ '%0.3f' %i[2].ra + " " + "&"+ " "+ "%0.3f" %(i[2].dec) +\
    " " + "&" + " " +i[2].fk5 +" " + '&' + "  "+ latexHREF +str(i[0])+"}" + "{%i}" %i[0] +" " + '&' + "  "+"\\" + "\\"
    
    lastname = i[1]

print "\\end{longtable}"

\begin{longtable}{c|c|c|c|c|c} 
\toprule 
Database & Ra (deg) & Dec (deg)& Fk5 & Id/Link & Candidate? (y/n)  \\\midrule 
\endfirsthead 
\toprule 
Database & Ra (deg) & Dec (deg) & Fk5 &Id/Link & Candidate? (y/n)  \\\midrule 
\endhead 
\bottomrule 
\endfoot 
\bottomrule
\caption{ Comparison position of candidates with different databases for the 1hr interval data} 
\endlastfoot
tgss \& vlssr & -175.262 & 50.444 & 12 18 57 +50 26 37 &  \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16306}{16306} &  \\
tgss \& vlssr & -174.211 & 44.590 & 12 23 09 +44 35 24 &  \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16326}{16326} &  \\
tgss \& vlssr & -171.721 & 45.481 & 12 33 07 +45 28 52 &  \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16375}{16375} &  \\
tgss \& vlssr & -173.524 & 44.939 & 12 25 54 +44 56 22 &  \href{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16130}{16130} &  \\
tgss \& 

In [578]:
data = lif.Reader('asu.fit')
data

,,.1,.2,.3,.4,[mJy],MHz,mJy,mJy.1,deg,deg.1,.5,.6,.7,arcsec
0,------,-,-------------------------------,--,------,------,-----,---------,--------,--------,--------,----,---,---------,---------
1,50608,0,GB6 B1220+5043,14,-1.16,6.26,4850,7.60e+01,1.5e+01,185.7117,+50.4492,zSim,SED,Radio+Opt,108.00
2,50608,0,87GB 122025.7+504330,14,-1.16,6.26,4850,6.70e+01,1.3e+01,185.7158,+50.4481,zSim,SED,Radio+Opt,138.00
3,50608,0,BWE 1220+5043,14,-1.16,6.26,4850,6.40e+01,1.3e+01,185.7183,+50.4472,zSim,SED,Radio+Opt,138.00
4,50608,0,4C +50.34,14,-1.16,6.26,178,4.40e+03,1.3e+03,185.7208,+50.4467,zSim,SED,Radio+Opt,690.00
5,50608,0,TXS 1220+507,14,-1.16,6.26,365,2.29e+03,4.6e+02,185.7163,+50.4486,zSim,SED,Radio+Opt,6.00
6,50608,0,WN B1220.4+5043,14,-1.11,6.14,325,2.57e+03,5.1e+02,185.7162,+50.4486,zSim,SED,Radio+Opt,12.00
7,50608,0,FIRST J122251.9+502655,14,-1.11,6.13,1400,4.50e+02,9.0e+01,185.7163,+50.4487,zSim,SED,Radio+Opt,5.00
8,50608,0,NVSS J122251+502655,14,-1.11,6.14,1400,4.47e+02,8.9e+01,185.7163,+50.4488,zSim,SED,Radio+Opt,45.00
9,50608,0,MY 122025.7+502701.2,14,-1.16,6.26,232,4.34e+03,8.7e+02,185.7158,+50.4503,zSim,SED,Radio+Opt,150.00


In [579]:

ra = data.deg[1:].astype(float)
flux = list(data.mJy[1:].astype(float))
freq = list(data.MHz[1:].astype(float))
fluxerr = list(data['mJy.1'][1:].astype(float))
print fluxerr
print flux
print freq

[15.0, 13.0, 13.0, 1300.0, 460.0, 510.0, 90.0, 89.0, 870.0, 80.0, 1100.0, 1000.0, 1100.0, 2100.0]
[76.0, 67.0, 64.0, 4400.0, 2290.0, 2570.0, 450.0, 447.0, 4340.0, 401.0, 5530.0, 5190.0, 5290.0, 10300.0]
[4850.0, 4850.0, 4850.0, 178.0, 365.0, 325.0, 1400.0, 1400.0, 232.0, 1400.0, 151.0, 151.0, 151.0, 74.0]


In [580]:
def starmodel(value,a,b,c):
    return a*value**-b+c

def starmodel2(value,*par):
    return 10**(-np.log(value)*par[0] + par[1])

def starmodel3(value,*par):
    return 10**(-np.log(value**par[0])*par[1] + par[2])

def ChiSq2(parm, xval, yval, dy): # the weighted least-squares
    ymod = starmodel2(xval,*parm)
    chisq = sum(pow((yval-ymod)/dy,2))
    return chisq
a=1
b = 10
c = 110
p0 = [a,b,c]  # Define starting values
p01=[b,c]
xval = freq
yval = flux
dy = fluxerr
# result = op.minimize(ChiSq2,p0,args=(xval,yval,dy),method="BFGS")
# print result
ml_cfpars, ml_cfcovar = op.curve_fit(starmodel2, xval, yval, p01, sigma=dy)
err = np.sqrt(np.diag(ml_cfcovar))
confidence = np.diag(ml_cfcovar)

In [581]:

logspace = np.linspace(0,1e4,500)
from scipy.stats import linregress

x = np.arange(0, 1e4, 1)
# print ml_cfpars
fit = starmodel2(x, *ml_cfpars)
# print fit
dof = len(xval)-len(p0)
chisquare = ChiSq2(ml_cfpars,xval,yval,dy)


/home/kmeulen/virtualenv/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


In [582]:
runcat = session.query(tkp.db.model.Runningcatalog).filter(tkp.db.model.Runningcatalog.id==runcat_id).one()

sources = runcat.extractedsources

y = [(s.ra +360) for s in sources]
x = [s.f_int*10**3 for s in sources]
xerr = [s.f_int_err*10**3 for s in sources]
h = [s.image.freq_eff/(1.*10**6) for s in sources]


d = figure(title="ligthcurve for runcat %s" % runcat.id, x_axis_label='freq', y_axis_label='flux',y_axis_type='log',x_axis_type='log')
# p.scatter([x, y],[flux,ra],color=['firebrick','navy'])

d.scatter(h,x,color = 'firebrick')


lower = []
upper = []
base =[]
for i in range(len((x))):
    lower.append(x[i]-xerr[i])
    upper.append(x[i]+xerr[i])
    base.append(h[i])

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

d.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)
lower = []
upper = []
base =[]
for i in range(len(flux)):
    lower.append(flux[i]-fluxerr[i])
    upper.append(flux[i]+fluxerr[i])
    base.append(freq[i])
# print freq
d.line(xfit,fit,color='green')

d.scatter(freq,flux,color ='navy')
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

d.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)

# print x
# p.xaxis.formatter = DatetimeTickFormatter()
show(d)
print "goodness of fit of:" +str(ss.chi2.sf(chisquare, df=dof))

goodness of fit of:0.588304972117089


In [583]:
print Transients.instances[0].pflux

7.18846941692


In [584]:
SavedPosSources(vars(Transients.instances[0]))

In [585]:
print PosVar.instances[0].pflux

1.11391900834


In [586]:
def reverseneg(ra):
    if ra<0:
        ra = ra +360
    return ra
pandacata['ra'] = pandacata.ra.apply(reverseneg)


In [587]:

filtered_data = pandacata[(182.1 <= pandacata.ra) & (pandacata.ra<=188.5) &(49.8 <=pandacata.dec)&(pandacata.dec<=51.1)]
filtered_data = filtered_data.sort_values(by=['ra'])


In [588]:
print len(tgsslist[0].ra)
print np.array(filtered_data.ra)

263
[182.61351015 182.61407722 182.61438567 182.6144716  182.61449014
 182.61460138 182.61460764 182.61471792 182.61476953 182.61477064
 182.61480203 182.61480323 182.61490213 182.6149171  182.61509948
 182.61510465 182.66645232 182.66733708 182.66745147 182.66759639
 182.66759816 182.66760779 182.66761549 182.66765279 182.66770232
 182.6677178  182.66776286 182.66783502 182.66798384 182.66807117
 182.66813148 182.66842532 182.91413814 182.91627254 182.9183627
 182.92057545 182.92097375 182.92164971 182.92220563 182.9223331
 182.92246259 182.92337161 182.92419529 182.92437765 182.93091833
 182.93849361 183.25740967 183.25754182 183.25800158 183.25821778
 183.25825845 183.25831106 183.2583148  183.25832814 183.25841071
 183.25841492 183.25843264 183.25845101 183.25850763 183.25855626
 183.25859354 183.25951657 183.28739094 183.28742126 183.28746523
 183.28778337 183.28819054 183.28824596 183.28862592 183.28863064
 183.28867576 183.28869649 183.28875756 183.2888362  183.28892221
 183.288

In [589]:
PosVar.instances = []
print type(filtered_data['ra'].astype(float))

CheckSimOutsideDatabase(tgsslist[0],filtered_data,rcheck = 3,ycheck = 5,\
                     checklight = False, gamma = True,Cali = False,deruiter = True,euccutoff = .06)

<class 'pandas.core.series.Series'>
before:
726


TypeError: string indices must be integers, not str

In [ ]:
x = [s for s in filtered_data.ra]
y = [s for s in filtered_data.pflux]
xt = [s for s in tgsslist[0].ra]
yt = [s for s in tgsslist[0].pflux]

p = figure(title="ligthcurve for runcat %s" % runcat.id, x_axis_label='ra', y_axis_label='flux', x_axis_type='log')
# p.scatter([x, y],[flux,ra],color=['firebrick','navy'])
p.line(xt,yt,color ='navy')
p.line(x,y,color = 'firebrick')
show(p)

# PIECE OF CODE WHERE WE TRIED TO EQUAL EUCDISTANCE TO RUITERDISTANCE VIA NUMERICAL INTEGRATIONS DIDNT WORK 

In [ ]:
# # The de Ruiter distance as shown in the Trap Paper section 4.4 https://arxiv.org/abs/1503.01526

# def CalcDeRuiterspec(ra1,ra1e,dec1,dec1e,ra2,ra2e,dec2,dec2e):
#     r=(np.sqrt((((-1*ra2 +ra1)**2)*(np.cos((dec1+dec2)/2))**2)/\
#                        (ra1e**2+ra2e**2)+((-1*dec2+dec1)**2/(dec1e**2+dec2e**2))))
#     return r 

In [ ]:
# def CompareEuc(data,racata,raecata,deccata,dececata,X):
    
#     ra = np.linspace(min(abs(racata)),max(abs(racata)),100)
#     dec = np.linspace(min(abs(np.array(deccata))),max(abs(np.array(deccata))),100)
#     rae = np.linspace(min(abs(raecata)),max(abs(raecata)),100)
#     dece = np.linspace(min(abs(np.array(dececata))),max(abs(np.array(dececata))),100)
    
# #   Initialize radec inorder to have the first value be a real value instead of radec = []
#     radeccomplete = None
#     radececomplete = None
    
#     for i in range(len(ra)):
#         for j in range(len(dec)):
#             if not radeccomplete:
#                 radeccomplete =[[ra[i],dec[j]]]
#                 radececomplete = [[rae[i],dece[j]]]
#             else:
#                 radeccomplete.append([ra[i],dec[j]])
#                 radececomplete.append([rae[i],dece[j]])

#     radece = np.matrix(radececomplete)
#     radec = np.matrix(radeccomplete)

# #   calculate the euler distance
#     Y = distance.cdist(radec,radec,'euclidean')

    
# #   Calculate all the deruiter distances
#     r = []
#     for i in range(len(radec)):
        
#         h = CalcDeRuiterspec(float(radec[i][:,0]),float(radece[i][:,0]),float(radec[i][:,1]),float(radece[i][:,0]),np.array(radec[:][:,0]),np.array(radece[:][:,0]),np.array(radec[:][:,1]),np.array(radece[:][:,1]))
#         r.append(h)

# #   Reshape the de ruiter distances in an Array compatible with the euclidean distance array
#     r = np.array(r)
#     r = np.reshape(r,(len(r),len(r)))
    
# #   combine the euclidean and the deruiter distances
# #     z = np.column_stack((Y,r))
#     return Y,r

                            



In [ ]:
# Y,r = CompareEuc(tgss,racata,raecata,deccata,dececata,X)

In [ ]:
# print r[368]
# print Y[368]

# print r[867]
# print Y[867]
# plt.scatter(r[368],Y[368])
# plt.scatter(r[867],Y[867])
# # print r[369][568]
# # print Y[369][568]

# # print r[867][4368]
# # print Y[867][4368]

In [ ]:
# # print r[0]
# # print Y[0]

# for i in range(len(r)):
#     for j in range(len(r[i])):
#         if Y[i][j] <=0.05 and Y[i][j]>0:
#             print i
#             print j
#             print Y[i][j]
#             print r[i][j]

# # stop = False
# # for i in range(len(Y)):
# #     for j in range(len(Y[i])):
# #         if r[i][j] -0.001 <= 5 <= r[i][j] +0.001:
# #             print Y[i][j]
# #             print r[i][j]
# #             print "______________"
# #     if stop:
# #         break
# # plt.scatter(Y,r)

In [ ]:
tgss = dl.ReadData("tgss")
print tgss

In [ ]:
for i in range(len(Transients.instances)):
    if Transients.instances[i].id == 3324:
        print i

In [ ]:
r = CompareFluxes(vars(Transients.instances[7]),np.array(lightcata),np.array(lightecata))

In [ ]:
for j in range(len(lightcata)):
    if 0.594<=lightcata[j]<=0.7143 :
        print j

for i in range(len(r)):
    if r[i] <=1:
        print r[i]
print 'yoloswag'
print r[103]
print r[225]
print r[601]

In [ ]:
print idcata[0]

Delete double entries in transient list

Delete candidates that have an euclidean distance measure of 0.04 or less with the rest of the sources